## TODOs:

1. bowler rank scraping
2. KL rahul name 

In [1]:
# import
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

pd.set_option('max_columns',100)


import os
os.chdir('/Users/shubhamjain/Downloads/AV/IPL/')

In [2]:
#xi = pd.read_csv('saved/playing_xi.csv')

In [2]:
## training data

ball_data = pd.read_csv('train/ball_by_ball_data.csv')
match_data = pd.read_csv('train/match_data.csv')
key_teams = pd.read_csv('train/key_teams.csv')
players = pd.read_csv('train/player_rosters.csv')
player_attributes = pd.read_csv('player_attributes.csv')

## test data

test_player = pd.read_csv('test/player_predictions.csv')
test_matches = pd.read_csv('test/matches_2018.csv')
test_extras = pd.read_csv('test/total_extras.csv')

## submission

sub_player = pd.read_csv('sample_submission/player_predictions.csv')
sub_extras = pd.read_csv('sample_submission/total_extras.csv')

In [3]:
## features

batsman = pd.read_csv('saved/batsman.csv')
bowler = pd.read_csv('saved/bowler.csv')

In [4]:
ids = test_player['player_id'].unique()

In [5]:
## treating KL rahul, b kumar, duminy

batsman.loc[batsman['batsman'] == 'B Kumar', 'player_id'] = 691
batsman.loc[batsman['batsman'] == 'J P Duminy', 'player_id'] = 677
batsman.loc[batsman['batsman'] == 'KL Rahul', 'player_id'] = 668

batsman.loc[batsman['batsman'] == 'B Kumar', 'country'] = 'India'
batsman.loc[batsman['batsman'] == 'J P Duminy', 'country'] = 'Africa'
batsman.loc[batsman['batsman'] == 'KL Rahul', 'country'] = 'India'

batsman.loc[batsman['batsman'] == 'B Kumar', 'Price'] = 850
batsman.loc[batsman['batsman'] == 'J P Duminy', 'Price'] = 100
batsman.loc[batsman['batsman'] == 'KL Rahul', 'Price'] = 1100

batsman.loc[batsman['batsman'] == 'B Kumar', 'role'] = 'Bowler'
batsman.loc[batsman['batsman'] == 'J P Duminy', 'role'] = 'Batsman'
batsman.loc[batsman['batsman'] == 'KL Rahul', 'role'] = 'WK-Batsman'

In [6]:
bowler.loc[bowler['bowler'] == 'B Kumar', 'player_id'] = 691
bowler.loc[bowler['bowler'] == 'J P Duminy', 'player_id'] = 677
bowler.loc[bowler['bowler'] == 'KL Rahul', 'player_id'] = 668

bowler.loc[bowler['bowler'] == 'B Kumar', 'country'] = 'India'
bowler.loc[bowler['bowler'] == 'J P Duminy', 'country'] = 'Africa'
bowler.loc[bowler['bowler'] == 'KL Rahul', 'country'] = 'India'

bowler.loc[bowler['bowler'] == 'B Kumar', 'Price'] = 850
bowler.loc[bowler['bowler'] == 'J P Duminy', 'Price'] = 100
bowler.loc[bowler['bowler'] == 'KL Rahul', 'Price'] = 1100

bowler.loc[bowler['bowler'] == 'B Kumar', 'role'] = 'Bowler'
bowler.loc[bowler['bowler'] == 'J P Duminy', 'role'] = 'bowler'
bowler.loc[bowler['bowler'] == 'KL Rahul', 'role'] = 'WK-bowler'

In [7]:
## save

batsman.to_csv('saved/new_batsman.csv', index=False)
bowler.to_csv('saved/new_bowler.csv', index=False)

In [8]:
batsman = batsman[batsman['player_id'].isin(ids)]
bowler = bowler[bowler['player_id'].isin(ids)]

In [9]:
bowler[bowler['Price'].isnull()]

,bowler,player_id,role,bowling_style,bowling_arm,Price,country,overs_bowled,consistency,wickets,attacking,maidan_overs,dot%,boundary%,strike_rate,orange_cap,prev_match_count,hauls,experience,Team_Consistency,mom,avg_first_ing,avg_second_ing,bharosa,bharosa2,Team
41,Ankit Rajput,471.0,Bowler,Medium,Right-Arm,NaN,NaN,25,7.680000,7.0,0.280000,NaN,0.453333,1.000000,4.666667,4.0,5.0,0.0,9,1.333333,0.0,0.500000,1.000000,0.222222,27.428571,KXI
44,Colin de Grandhomme,610.0,Batting Allrounder,Fast-Medium,Right-Arm,NaN,NaN,17,7.588235,4.0,0.235294,NaN,0.382353,0.941176,3.921569,4.0,9.0,0.0,9,1.000000,0.0,0.142857,1.500000,0.111111,32.250000,RCB
58,Shakib Al Hasan,359.0,Batting Allrounder,Orthodox,Left-Arm,NaN,NaN,48,8.083333,9.0,0.187500,NaN,0.385417,1.437500,3.125000,NaN,1.0,0.0,14,4.000000,0.0,0.714286,0.571429,0.500000,43.111111,SRH
61,Washington Sunder,631.0,Batting Allrounder,Offbreak,Right-Arm,NaN,NaN,30,6.100000,8.0,0.266667,NaN,0.483333,0.766667,4.444444,8.0,10.0,0.0,10,1.000000,1.0,0.600000,1.000000,0.300000,22.875000,RCB
78,Gurkeerat Singh Mann,443.0,Batsman,Offbreak,Right-Arm,NaN,NaN,13,7.153846,5.0,0.384615,NaN,0.346154,1.461538,6.410256,NaN,NaN,0.0,6,3.500000,0.0,0.500000,1.000000,0.000000,18.600000,DD
82,Pradeep Sahu,664.0,Batting Allrounder,Break,Leg,NaN,NaN,18,8.000000,3.0,0.166667,NaN,0.361111,1.055556,2.777778,NaN,NaN,0.0,5,2.000000,0.0,NaN,1.000000,0.600000,48.000000,KXI


In [10]:
bowler[bowler['bowler'] == 'Colin de Grandhomme']

,bowler,player_id,role,bowling_style,bowling_arm,Price,country,overs_bowled,consistency,wickets,attacking,maidan_overs,dot%,boundary%,strike_rate,orange_cap,prev_match_count,hauls,experience,Team_Consistency,mom,avg_first_ing,avg_second_ing,bharosa,bharosa2,Team
44,Colin de Grandhomme,610.0,Batting Allrounder,Fast-Medium,Right-Arm,NaN,NaN,17,7.588235,4.0,0.235294,NaN,0.382353,0.941176,3.921569,4.0,9.0,0.0,9,1.0,0.0,0.142857,1.5,0.111111,32.25,RCB


In [11]:
##price and country

batsman.loc[batsman['batsman'] == 'Surya Kumar Yadav', 'Price'] = 320
batsman.loc[batsman['batsman'] == 'Faf du Plessis', 'Price'] = 160
batsman.loc[batsman['batsman'] == 'Mahendra Singh Dhoni', 'Price'] = 1500
batsman.loc[batsman['batsman'] == 'Steven Smith', 'Price'] = 1200
batsman.loc[batsman['batsman'] == 'Shakib Al Hasan', 'Price'] = 200
batsman.loc[batsman['batsman'] == 'AB De Villiers', 'Price'] = 1100
batsman.loc[batsman['batsman'] == 'Gurkeerat Singh Mann', 'Price'] = 75
batsman.loc[batsman['batsman'] == 'Quinton de Kock', 'Price'] = 280
batsman.loc[batsman['batsman'] == 'Pradeep Sahu', 'Price'] = 20
batsman.loc[batsman['batsman'] == 'Colin de Grandhomme', 'Price'] = 220
batsman.loc[batsman['batsman'] == 'Washington Sunder', 'Price'] = 320
batsman.loc[batsman['batsman'] == 'Ankit Rajput', 'Price'] = 220


batsman.loc[batsman['batsman'] == 'Surya Kumar Yadav', 'country'] = 'India'
batsman.loc[batsman['batsman'] == 'Faf du Plessis', 'country'] = 'Africa'
batsman.loc[batsman['batsman'] == 'Mahendra Singh Dhoni', 'country'] = 'India'
batsman.loc[batsman['batsman'] == 'Steven Smith', 'country'] = 'Australia'
batsman.loc[batsman['batsman'] == 'Shakib Al Hasan', 'country'] = 'Bangladesh'
batsman.loc[batsman['batsman'] == 'AB De Villiers', 'country'] = 'Africa'
batsman.loc[batsman['batsman'] == 'Gurkeerat Singh Mann', 'country'] = 'India'
batsman.loc[batsman['batsman'] == 'Quinton de Kock', 'country'] = 'Africa'
batsman.loc[batsman['batsman'] == 'Pradeep Sahu', 'country'] = 'India'
batsman.loc[batsman['batsman'] == 'Colin de Grandhomme', 'country'] = 'Zealand'
batsman.loc[batsman['batsman'] == 'Washington Sunder', 'country'] = 'India'
batsman.loc[batsman['batsman'] == 'Ankit Rajput', 'country'] = 'India'

In [12]:
batsman['country'].unique()

array(['Australia', 'India', 'Zealand', 'Windies', 'Africa', 'Bangladesh',
       'England', 'Afghanistan'], dtype=object)

In [13]:
batsman['role'].value_counts()

Bowler                43
Batsman               36
WK-Batsman            17
Batting Allrounder    15
Bowling Allrounder    14
Name: role, dtype: int64

In [14]:
## removing players who won't be attending

not_playing = ['Mitchell Santner', 'Mitchell Starc', 'Jason Behrendorff', 'Steven Smith', 'Nathan Coulter-Nile',
              '	David Warner']

batsman = batsman[~batsman['batsman'].isin(not_playing)]
bowler = bowler[~bowler['bowler'].isin(not_playing)]

In [15]:
## adding team name in batsman 

df = players[players['Season'] == 2018][['player_id','Team']]

batsman = batsman.merge(df, on ='player_id', how='left')

In [16]:
## minor correction

batsman.loc[batsman['batsman'] == 'AB De Villiers', 'role'] = 'Batsman'
batsman.loc[batsman['batsman'] == 'Robin Uthappa', 'role'] = 'Batsman'
batsman.loc[batsman['batsman'] == 'Jos Buttler', 'role'] = 'Batsman'

### selecting top 4 batsman

In [17]:
temp = batsman[batsman['role'] == 'Batsman'].sort_values(['Team','Average'],ascending=False).groupby(['Team']).head(4)

In [18]:
batting = temp[['Team','batsman','role','country']]

In [19]:
batting = batting.rename(columns = {'batsman':'player'})

### selecting 1 wk

In [20]:
temp = batsman[batsman['role'] == 'WK-Batsman'].sort_values(['Team','Price'],ascending=False).groupby(['Team']).head(1)

In [21]:
wk = temp[['Team','batsman','role','country']]

In [22]:
wk = wk.rename(columns = {'batsman':'player'})

### selecting 3 bowlers

In [23]:
## adding team name in batsman 

df = players[players['Season'] == 2018][['player_id','Team']]

bowler = bowler.merge(df, on ='player_id', how='left')

In [32]:
bowler.drop(['Team_y'],axis=1,inplace=True)
bowler = bowler.rename(columns ={'Team_x':'Team'})

In [33]:
temp = bowler[bowler['role'] == 'Bowler'].sort_values(['Team','Price'],ascending=False).groupby(['Team']).head(3)

In [34]:
bowling = temp[['Team','bowler','role','country']]

In [35]:
bowling = bowling.rename(columns = {'bowler':'player'})

###  selecting 3 all rounder

In [36]:
temp = batsman[(batsman['role'] == 'Batting Allrounder') | 
              (batsman['role'] == 'Bowling Allrounder')].sort_values(['Team','Average'],ascending=False).groupby(['Team']).head(3)

In [37]:
all_round = temp[['Team','batsman','role','country']]

In [38]:
all_round = all_round.rename(columns = {'batsman':'player'})

### playing xi

In [39]:
playing_xi = pd.concat([batting,wk,all_round,bowling]).sort_values(['Team','role'])

In [46]:
playing_xi[playing_xi['Team'] == 'MI']

,Team,player,role,country
3,MI,Rohit Sharma,Batsman,India
7,MI,Surya Kumar Yadav,Batsman,India
19,MI,J P Duminy,Batsman,Africa
70,MI,Saurabh Tiwary,Batsman,India
84,MI,Krunal Pandya,Batting Allrounder,India
53,MI,Kieron Pollard,Batting Allrounder,Windies
14,MI,Jasprit Bumrah,Bowler,India
32,MI,Pat Cummins,Bowler,Australia
42,MI,Mustafizur Rahman,Bowler,Bangladesh
96,MI,Hardik Pandya,Bowling Allrounder,India


In [41]:
## manual tweaks in playing xi

playing_xi.loc[playing_xi['player'] == 'Karn Sharma','player'] = 'Ravindra Jadeja'

playing_xi.loc[playing_xi['player'] == 'Jason Roy','country'] = 'Zealand'
playing_xi.loc[playing_xi['player'] == 'Jason Roy','player'] = 'Colin Munro'


playing_xi.loc[playing_xi['player'] == 'Daniel Christian','role'] = 'Bowler'
playing_xi.loc[playing_xi['player'] == 'Daniel Christian','player'] = 'Mohammed Shami'

playing_xi.loc[playing_xi['player'] == 'Piyush Chawla','player'] = 'Vinay Kumar'

playing_xi.loc[playing_xi['player'] == 'Ben Cutting','country'] = 'India'
playing_xi.loc[playing_xi['player'] == 'Ben Cutting','player'] = 'Hardik Pandya'

In [42]:
playing_xi.loc[playing_xi['player'] == 'David Warner','country'] = 'India'
playing_xi.loc[playing_xi['player'] == 'David Warner','player'] = 'Shikhar Dhawan'

In [43]:
playing_xi.loc[playing_xi['player'] == 'Kagiso Rabada','country'] = 'Zealand'
playing_xi.loc[playing_xi['player'] == 'Kagiso Rabada','player'] = 'Trent Boult'

### players left

1. 1 in KKR -- kamlesh nagarkoti
2. 2 in RR  -- Heinrich Klaasen, Ben Laughlin

In [44]:
playing_xi.shape

(85, 4)

In [215]:
playing_xi.to_csv('saved/playing_xi.csv', index=False)

In [204]:
batsman.to_csv('saved/final_batsman.csv', index=False)
bowler.to_csv('saved/final_bowler.csv', index=False)

In [211]:
batsman[batsman['batsman'] == 'Kagiso Rabada']

,batsman,Average,Season_played,total_runs,100s,50s,30+,total_innings,ability,dot%,power_hitting,consistency,avg_first_ing,avg_second_ing,CSK,DD,GL,KKR,KXI,MI,RCB,RPS,RR,SRH,top_order,Team_Consistency,current_rank,Price,country,last_season_runs,role,mom,player_id,Team
114,Kagiso Rabada,80.882353,2.0,55,0,0,1,3,0.333333,51.470588,0.088235,18.333333,11.0,22.0,0.0,0.0,0.0,0.0,11.0,22.0,0.0,0.0,0.0,0.0,0,2.0,999.0,420.0,Africa,55.0,Bowler,0.0,594,DD


In [216]:
bowler[bowler['bowler'] == 'Trent Boult']

,bowler,player_id,role,bowling_style,bowling_arm,Price,country,overs_bowled,consistency,wickets,attacking,maidan_overs,dot%,boundary%,strike_rate,orange_cap,prev_match_count,hauls,experience,Team_Consistency,mom,avg_first_ing,avg_second_ing,bharosa,bharosa2,Team
17,Trent Boult,565,Bowler,Fast-Medium,Left-Arm,220.0,Zealand,56,8.392857,15.0,0.267857,NaN,0.470238,1.214286,4.464286,5.0,6.0,0.0,14,1.333333,1.0,1.125,1.0,1.0,31.333333,DD


In [7]:
xi[xi['Team'] == 'RR']

,Team,player,role,country
65,RR,Rahul Tripathi,Batsman,India
66,RR,Jos Buttler,Batsman,England
67,RR,Ajinkya Rahane,Batsman,India
68,RR,Stuart Binny,Batting Allrounder,India
69,RR,Jaydev Unadkat,Bowler,India
70,RR,Dhawal Kulkarni,Bowler,India
71,RR,Anureet Singh,Bowler,India
72,RR,Ben Stokes,Bowling Allrounder,England
73,RR,Sanju Samson,WK-Batsman,India
